In [5]:
import geopandas as gpd
import pandas as pd
from glob import glob

In [3]:
kdprov=['51']
nmfiledegurba="degurba_Bali_00.gpkg"

In [4]:
data_degurba_grod=gpd.read_file('data/output/2_grid_degurba/'+nmfiledegurba)

In [7]:
data_pop=gpd.GeoDataFrame()
for i in kdprov:
    gl_pop=glob("data/output/1_population_density/grid_pop_"+i+"*_100m.gpkg")
    for j in gl_pop:
        y_temp=gpd.read_file(j)
        y_temp['geometry']=y_temp.centroid
        y_temp=y_temp[['j_pend','geometry','p_area']]
        data_pop=y_temp.append(y_temp)

/tmp/ipykernel_6601/2622914391.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_pop=y_temp.append(y_temp)
/tmp/ipykernel_6601/2622914391.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_pop=y_temp.append(y_temp)
/tmp/ipykernel_6601/2622914391.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_pop=y_temp.append(y_temp)
/tmp/ipykernel_6601/2622914391.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_pop=y_temp.append(y_temp)
/tmp/ipykernel_6601/2622914391.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_pop=y_temp.

In [9]:
data_desa_raster=gpd.GeoDataFrame()
for i in kdprov:
    temp=gpd.read_file('data/output/0_raster_data_desa/'+'desa_pend_rasterize_'+i+'.gpkg')
    data_desa_raster=temp.append(data_desa_raster)

/tmp/ipykernel_6601/2837293180.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_desa_raster=temp.append(data_desa_raster)


In [12]:
pop_w_grid=data_pop.sjoin(data_degurba_grod)[['j_pend','classification','geometry']]

In [18]:
data_desa=data_desa_raster.sjoin(pop_w_grid)[['j_pend','classification','iddesa','geometry']]

In [19]:
data_desa

,j_pend,classification,iddesa,geometry
375,0.000000,RURAL,5106010001,"POLYGON ((11480400.000 -1045300.000, 11480500...."
375,0.000000,CENTRE,5106010001,"POLYGON ((11480400.000 -1045300.000, 11480500...."
375,0.000000,CENTRE,5106010001,"POLYGON ((11480400.000 -1045300.000, 11480500...."
375,0.000000,CENTRE,5106010001,"POLYGON ((11480400.000 -1045300.000, 11480500...."
375,0.000000,CENTRE,5106010001,"POLYGON ((11480400.000 -1045300.000, 11480500...."
...,...,...,...,...
672,72.318117,CENTRE,5108090010,"POLYGON ((11495200.000 -1006600.000, 11495700...."
672,0.000000,CENTRE,5108090010,"POLYGON ((11495200.000 -1006600.000, 11495700...."
672,0.000000,CENTRE,5108090010,"POLYGON ((11495200.000 -1006600.000, 11495700...."
672,0.000000,CENTRE,5108090010,"POLYGON ((11495200.000 -1006600.000, 11495700...."


In [20]:
tab_data=pd.pivot_table(data_desa,index='iddesa',columns='classification',
                        values='j_pend',aggfunc='sum').fillna(0).reset_index()

In [21]:
def define_LAU(y):
    total_pop=y.CENTRE+y.CLUSTER+y.RURAL
    if total_pop==0:
        return 'RURALS'
    elif (y.CENTRE/total_pop)>=0.5:
        return 'CITIES'
    elif (((y.CENTRE/total_pop)<0.5)&((y.RURAL/total_pop)<0.5)):
        return 'TOWNS'
    else:
        return 'RURALS'

In [22]:
tab_data['LAU']=tab_data.apply(lambda y:define_LAU(y),axis=1)

In [24]:
peta_desa=gpd.read_file(r"data/input/Master_Data/DESA_20201_BYINDO.gpkg").query('kdprov in @kdprov')

In [25]:
peta_desa_LAU=peta_desa.merge(tab_data[['iddesa','LAU']],how='left').fillna('RURALS')

In [27]:
for i in kdprov:
    peta_desa_LAU.query("kdprov==@i").to_file(
        'data/output/3_small_spatial_unit_degurba/small_spatial_unit_'+i+'.gpkg',driver='GPKG')

In [28]:
from dbfread import DBF
mfd_=pd.DataFrame(iter(DBF('data/input/Master_Data/mfd_2020_1.dbf')))

In [29]:
mfd_['iddesa']=mfd_.PROP+mfd_.KAB+mfd_.KEC+mfd_.DESA

In [31]:
for i in kdprov:
    peta_desa_LAU.merge(mfd_[['iddesa','DRH']]).query("kdprov==@i").to_file(
        'data/output/3_small_spatial_unit_degurba/small_spatial_unit_'+i+'.gpkg',driver='GPKG')

In [33]:
peta_desa_LAU.merge(mfd_[['iddesa','DRH']]).pivot_table(index='DRH',columns='LAU',values='iddesa',aggfunc='count').fillna(0)

LAU,CITIES,RURALS,TOWNS
DRH,,,
1,0.0,260.0,9.0
2,2.0,393.0,52.0
